In [45]:
%%writefile pass-crack.c
#define _GNU_SOURCE
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <crypt.h>
#include <pthread.h>
#include <stdatomic.h>
#include <unistd.h>
#include <time.h>
#include <ctype.h>

typedef unsigned long long ull;
#define TOTAL_COMBINATIONS (26ULL * 26ULL * 100ULL)

typedef struct {
    int tid;
    ull start;
    ull end;
    const char *encrypted;
} targs_t;

static atomic_int found_flag = 0;
static char found_pass[8] = {0};
static pthread_mutex_t found_mutex = PTHREAD_MUTEX_INITIALIZER;
static _Atomic unsigned long long attempts = 0ULL;

static void extract_salt(const char *encrypted, char *saltbuf, size_t bufsz) {
    const char *p = encrypted;
    int d = 0;
    size_t i = 0;
    while (*p != '\0' && i + 1 < bufsz) {
        saltbuf[i++] = *p;
        if (*p == '$') {
            d++;
            if (d == 3) break;
        }
        p++;
    }
    saltbuf[i] = '\0';
    if (i == 0) {
        strncpy(saltbuf, encrypted, bufsz - 1);
        saltbuf[bufsz - 1] = '\0';
    }
}

static void idx_to_pass(char out[5], ull idx) {
    ull per_first = 26ULL * 100ULL; // 2600
    ull first = idx / per_first;
    ull rem = idx % per_first;
    ull second = rem / 100ULL;
    ull number = rem % 100ULL;
    out[0] = 'A' + (char)first;
    out[1] = 'A' + (char)second;
    out[2] = '0' + (char)(number / 10);
    out[3] = '0' + (char)(number % 10);
    out[4] = '\0';
}

static void *worker(void *arg) {
    targs_t *a = (targs_t*)arg;
    struct crypt_data data;
    data.initialized = 0;
    char salt[128];
    extract_salt(a->encrypted, salt, sizeof(salt));
    char cand[8];

    for (ull i = a->start; i <= a->end; ++i) {
        if (atomic_load(&found_flag)) break;
        idx_to_pass(cand, i);
        char *res = crypt_r(cand, salt, &data);
        atomic_fetch_add_explicit(&attempts, 1ULL, memory_order_relaxed);
        if (strcmp(res, a->encrypted) == 0) {
            pthread_mutex_lock(&found_mutex);
            if (!atomic_load(&found_flag)) {
                strncpy(found_pass, cand, sizeof(found_pass)-1);
                atomic_store(&found_flag, 1);
            }
            pthread_mutex_unlock(&found_mutex);
            break;
        }
    }
    free(a);
    return NULL;
}

char* generate_encrypted_password_c(const char* password, const char* salt, const char* algorithm) {
    char full_salt[256]; // Buffer for the full salt string
    const char* salt_prefix = "";

    if (algorithm == NULL || strcmp(algorithm, "SHA512") == 0) {
        salt_prefix = "$6$";
    } else if (strcmp(algorithm, "SHA256") == 0) {
        salt_prefix = "$5$";
    } else if (strcmp(algorithm, "MD5") == 0) {
        salt_prefix = "$1$";
    } else {
        fprintf(stderr, "Unsupported algorithm: %s\n", algorithm);
        return NULL; // Indicate error
    }

    if (salt == NULL) {
        char salt_chars[] = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789";
        size_t salt_len = strlen(salt_chars);
        size_t rand_salt_len = 16;
        if (strcmp(algorithm, "MD5") == 0) rand_salt_len = 8;
        if (strlen(salt_prefix) + rand_salt_len + 1 > sizeof(full_salt)) {
             fprintf(stderr, "Salt buffer too small\n");
             return NULL;
        }

        strcpy(full_salt, salt_prefix);
        srand(time(NULL)); // Seed random number generator
        for (size_t i = 0; i < rand_salt_len; ++i) {
            full_salt[strlen(full_salt)] = salt_chars[rand() % salt_len];
        }
        full_salt[strlen(full_salt)] = '\0';

    } else {
        // Use the provided salt
        if (strlen(salt_prefix) + strlen(salt) + 1 > sizeof(full_salt)) {
             fprintf(stderr, "Salt buffer too small\n");
             return NULL;
        }
        strcpy(full_salt, salt_prefix);
        strcat(full_salt, salt);
    }

    // crypt() returns a pointer to a static buffer, subsequent calls overwrite it.
    // If you need to store multiple encrypted passwords, you'd need to copy the result.
    return crypt(password, full_salt);
}


int main(int argc, char **argv) {
    printf("argc: %d\n", argc); // Debug print
    const char *encrypted;
    if (argc < 2) {
        printf("Using demo password logic.\n"); // Debug print
        encrypted = generate_encrypted_password_c("AB12", NULL, "SHA512");
        if (encrypted == NULL) {
            fprintf(stderr, "Error generating demo password.\n");
            return 1;
        }
        printf("Using demo password 'AB12', encrypted: %s\n", encrypted);
    } else {
        printf("Using provided encrypted password.\n"); // Debug print
        encrypted = argv[1];
    }

    int nthreads = 0;
    if (argc >= 3) {
        nthreads = atoi(argv[2]);
        if (nthreads <= 0) nthreads = 1;
    } else {
        long p = sysconf(_SC_NPROCESSORS_ONLN);
        nthreads = (p > 0) ? (int)p : 4;
    }

    ull total = TOTAL_COMBINATIONS;
    ull base = total / (ull)nthreads;
    ull rem = total % (ull)nthreads;

    pthread_t *threads = malloc(sizeof(pthread_t) * nthreads);
    if (!threads) { perror("malloc"); return 2; }

    ull next = 0;
    printf("Using %d threads. Total combinations: %llu\n", nthreads, (unsigned long long)total);
    for (int i = 0; i < nthreads; ++i) {
        ull cnt = base + (i < (int)rem ? 1ULL : 0ULL);
        ull start = next;
        ull end = (cnt==0) ? (start > 0 ? start-1 : 0) : (start + cnt - 1);
        next = start + cnt;
        targs_t *t = malloc(sizeof(targs_t));
        if (!t) { perror("malloc targ"); free(threads); return 3; }
        t->tid = i;
        t->start = start;
        t->end = end;
        t->encrypted = encrypted;
        if (pthread_create(&threads[i], NULL, worker, t) != 0) {
            fprintf(stderr, "Failed to create thread %d\n", i);
            free(t);
            continue;
        }
        printf("Thread %2d -> [%6llu .. %6llu]\n", i, (unsigned long long)start, (unsigned long long)end);
    }

    time_t st = time(NULL);
    for (int i = 0; i < nthreads; ++i) {
        pthread_join(threads[i], NULL);
    }
    time_t en = time(NULL);
    double secs = difftime(en, st);

    if (atomic_load(&found_flag)) {
        printf("Found: %s\n", found_pass);
    } else {
        printf("Not found in search space.\n");
    }
    printf("Attempts: %llu, Time: %.2fs\n",
           (unsigned long long)atomic_load_explicit(&attempts, memory_order_relaxed), secs);
    free(threads);
    return 0;
}

Overwriting pass-crack.c


In [46]:
%%bash
gcc -std=c11 -O2 -pthread -o pass-crack pass-crack.c -lcrypt
echo "Compile status: $?"

Compile status: 0


In [47]:
%%bash
./pass-crack

argc: 1
Using demo password logic.
Using demo password 'AB12', encrypted: $6$n875ygQGHSdJhli2$R2/jzKzfsLy93tovU/TfOdGuMx3Mo9N1js463/YbcTwB2kL01Ajh0L2/cHlnNxxI3yGRsq/BdoGwpDiVX0RR31
Using 2 threads. Total combinations: 67600
Thread  0 -> [     0 ..  33799]
Thread  1 -> [ 33800 ..  67599]
Found: AB12
Attempts: 236, Time: 1.00s
